<a href="https://colab.research.google.com/github/Muhammadsulton1/ML/blob/main/Icdar15_for_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2

import math
import copy
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import os
import time
import copy
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms  

import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
def statistics_label_cnt(lbl_path, lbl_cnt_map):
    """
    统计标签文件中都包含哪些label以及各自出现的次数
    """
    with open(lbl_path, 'r') as reader:
        for line in reader:
            items = line.rstrip().split(',')
            img_name = items[0]
            lbl_str = items[1].strip()[1:-1]
            for lbl in lbl_str:
                if lbl not in lbl_cnt_map.keys():
                    lbl_cnt_map[lbl] = 1
                else:
                    lbl_cnt_map[lbl] += 1


def statistics_max_len_label(lbl_path):
    """
    统计标签文件中最长的label所包含的字符数
    """
    max_len = -1
    with open(lbl_path, 'r') as reader:
        for line in reader:
            items = line.rstrip().split(',')
            img_name = items[0]
            lbl_str = items[1].strip()[1:-1]
            lbl_len = len(lbl_str)
            max_len = max_len if max_len > lbl_len else lbl_len
    return max_len


def load_lbl2id_map(lbl2id_map_path):
    """
    读取label-id映射关系记录文件
    """
    lbl2id_map = dict()
    id2lbl_map = dict()
    with open(lbl2id_map_path, 'r') as reader:
        for line in reader:
            items = line.rstrip().split('\t')
            label = items[0]
            cur_id = int(items[1])
            lbl2id_map[label] = cur_id
            id2lbl_map[cur_id] = label
    return lbl2id_map, id2lbl_map

In [3]:
base_data_dir = '/content/drive/MyDrive/icdar15'

train_img_dir = os.path.join(base_data_dir, 'train')
valid_img_dir = os.path.join(base_data_dir, 'test')
train_lbl_path = os.path.join(base_data_dir, 'gt.txt')
valid_lbl_path = os.path.join(base_data_dir, 'test_gt.txt')
lbl2id_map_path = os.path.join(base_data_dir, 'lbl2id_map.txt')

# 统计数据集中出现的所有的label中包含字符最多的有多少字符
train_max_label_len = statistics_max_len_label(train_lbl_path)
valid_max_label_len = statistics_max_len_label(valid_lbl_path)
max_label_len = max(train_max_label_len, valid_max_label_len)
print(f"数据集中包含字符最多的label长度为{max_label_len}")

# 统计数据集中出现的所有的符号
lbl_cnt_map = dict()
statistics_label_cnt(train_lbl_path, lbl_cnt_map)
print("训练集中出现的label")
print(lbl_cnt_map)
statistics_label_cnt(valid_lbl_path, lbl_cnt_map)
print("训练集+验证集中出现的label")
print(lbl_cnt_map)

数据集中包含字符最多的label长度为21
训练集中出现的label
{'G': 352, 'e': 1080, 'n': 623, 'a': 858, 'x': 28, 'i': 666, 's': 577, ' ': 54, 'T': 918, 'h': 308, 't': 582, 'r': 664, '[': 2, '0': 185, '6': 39, ']': 2, '2': 120, '-': 69, '3': 50, 'C': 606, 'p': 203, 'k': 100, 'E': 1474, 'X': 113, 'I': 879, 'R': 861, 'f': 141, 'u': 298, 'o': 685, 'l': 427, 'v': 123, 'A': 1229, 'U': 333, 'O': 1007, 'N': 805, 'c': 331, 'm': 216, 'W': 185, 'H': 408, 'Y': 237, 'P': 406, 'F': 269, '?': 5, 'S': 1199, 'b': 92, 'g': 173, 'L': 768, 'M': 375, 'D': 394, 'd': 259, '$': 47, '5': 76, '4': 44, '.': 96, 'w': 99, 'B': 336, '1': 184, '7': 44, '8': 44, 'V': 161, 'y': 165, 'K': 171, '!': 53, '9': 66, 'z': 12, ';': 3, '#': 16, 'j': 15, "'": 51, 'J': 77, ':': 19, '%': 28, '/': 24, 'q': 3, 'Q': 19, '(': 6, ')': 5, '\\': 8, '"': 8, '´': 3, 'Z': 29, '&': 9, 'É': 1, '@': 4, '=': 1, '+': 1}
训练集+验证集中出现的label
{'G': 536, 'e': 1575, 'n': 910, 'a': 1230, 'x': 39, 'i': 948, 's': 816, ' ': 86, 'T': 1353, 'h': 431, 't': 852, 'r': 957, '[': 2, '0': 2

In [4]:
train_max_label_len = statistics_max_len_label(train_lbl_path)
valid_max_label_len = statistics_max_len_label(valid_lbl_path)
max_label_len = max(train_max_label_len, valid_max_label_len)
print(f"数据集中包含字符最多的label长度为{max_label_len}")

# 统计数据集中出现的所有的符号
lbl_cnt_map = dict()
statistics_label_cnt(train_lbl_path, lbl_cnt_map)
print("训练集中出现的label")
print(lbl_cnt_map)
statistics_label_cnt(valid_lbl_path, lbl_cnt_map)
print("训练集+验证集中出现的label")
print(lbl_cnt_map)

# 构造 label - id 之间的映射
print("\n\n构造 label - id 之间的映射")
lbl2id_map = dict()
# 初始化两个特殊字符
lbl2id_map['☯'] = 0    # padding标识符
lbl2id_map['■'] = 1    # 句子起始符
lbl2id_map['□'] = 2    # 句子结束符
# 生成其余label的id映射关系
cur_id = 3
for lbl in lbl_cnt_map.keys():
    lbl2id_map[lbl] = cur_id
    cur_id += 1
# 保存 label - id 之间的映射
with open(lbl2id_map_path, 'w') as writer:
    for lbl in lbl2id_map.keys():
        cur_id = lbl2id_map[lbl]
        print(lbl, cur_id)
        line = lbl + '\t' + str(cur_id) + '\n'
        writer.write(line)

# 分析数据集图片尺寸

数据集中包含字符最多的label长度为21
训练集中出现的label
{'G': 352, 'e': 1080, 'n': 623, 'a': 858, 'x': 28, 'i': 666, 's': 577, ' ': 54, 'T': 918, 'h': 308, 't': 582, 'r': 664, '[': 2, '0': 185, '6': 39, ']': 2, '2': 120, '-': 69, '3': 50, 'C': 606, 'p': 203, 'k': 100, 'E': 1474, 'X': 113, 'I': 879, 'R': 861, 'f': 141, 'u': 298, 'o': 685, 'l': 427, 'v': 123, 'A': 1229, 'U': 333, 'O': 1007, 'N': 805, 'c': 331, 'm': 216, 'W': 185, 'H': 408, 'Y': 237, 'P': 406, 'F': 269, '?': 5, 'S': 1199, 'b': 92, 'g': 173, 'L': 768, 'M': 375, 'D': 394, 'd': 259, '$': 47, '5': 76, '4': 44, '.': 96, 'w': 99, 'B': 336, '1': 184, '7': 44, '8': 44, 'V': 161, 'y': 165, 'K': 171, '!': 53, '9': 66, 'z': 12, ';': 3, '#': 16, 'j': 15, "'": 51, 'J': 77, ':': 19, '%': 28, '/': 24, 'q': 3, 'Q': 19, '(': 6, ')': 5, '\\': 8, '"': 8, '´': 3, 'Z': 29, '&': 9, 'É': 1, '@': 4, '=': 1, '+': 1}
训练集+验证集中出现的label
{'G': 536, 'e': 1575, 'n': 910, 'a': 1230, 'x': 39, 'i': 948, 's': 816, ' ': 86, 'T': 1353, 'h': 431, 't': 852, 'r': 957, '[': 2, '0': 2

In [5]:
print("\n\n 分析数据集图片尺寸")
min_h = 1e10
min_w = 1e10
max_h = -1
max_w = -1
min_ratio = 1e10
max_ratio = 0
for img_name in os.listdir(train_img_dir):
    img_path = os.path.join(train_img_dir, img_name)
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    ratio = w / h
    min_h = min_h if min_h <= h else h
    max_h = max_h if max_h >= h else h
    min_w = min_w if min_w <= w else w 
    max_w = max_w if max_w >= w else w 
    min_ratio = min_ratio if min_ratio <= ratio else ratio
    max_ratio = max_ratio if max_ratio >= ratio else ratio
print("min_h", min_h)
print("max_h", max_h)
print("min_w", min_w)
print("max_w", max_w)
print("min_ratio", min_ratio)
print("max_ratio", max_ratio)



 分析数据集图片尺寸
min_h 9
max_h 658
min_w 15
max_w 628
min_ratio 0.09302325581395349
max_ratio 8.619047619047619


In [6]:
# Model Architecture
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. 
    Base for this and many other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed    # input embedding module(input embedding + positional encode)
        self.tgt_embed = tgt_embed    # ouput embedding module
        self.generator = generator    # output generation module
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        memory = self.encode(src, src_mask)
        res = self.decode(memory, src_mask, tgt, tgt_mask)
        return res
    
    def encode(self, src, src_mask):
        src_embedds = self.src_embed(src)
        return self.encoder(src_embedds, src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        target_embedds = self.tgt_embed(tgt)
        return self.decoder(target_embedds, memory, src_mask, tgt_mask)


class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)


def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class Encoder(nn.Module):
    """
    Encoder
    The encoder is composed of a stack of N=6 identical layers.
    """
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

# We employ a residual connection around each of the two sub-layers, followed by layer normalization
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, feature_size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(feature_size))
        self.b_2 = nn.Parameter(torch.zeros(feature_size))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        sublayer_out = sublayer(x)
        sublayer_out = self.dropout(sublayer_out)
        x_norm = x + self.norm(sublayer_out)
        return x_norm


class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size   # embedding's dimention of model, 默认512

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        # attention sub layer
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        # feed forward sub layer
        z = self.sublayer[1](x, self.feed_forward)
        return z


# Decoder
# The decoder is also composed of a stack of N=6 identical layers.

class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)


def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


# Attention
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))


# Embeddings and Softmax
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        embedds = self.lut(x)
        return embedds * math.sqrt(self.d_model)    # TODO 这里的归一化操作的目的?

# Positional Encoding
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # TODO 位置信息编码的具体原理
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)


# Full Model
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    """
    构建模型
    params:
        src_vocab:
        tgt_vocab:
        N: 编码器和解码器堆叠基础模块的个数
        d_model: 模型中embedding的size，默认512
        d_ff: FeedForward Layer层中embedding的size，默认2048
        h: MultiHeadAttention中多头的个数，必须被d_model整除
        dropout:
    """
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [11]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        """
        norm: loss的归一化系数，用batch中所有有效token数即可
        """
        x = self.generator(x)
        x_ = x.contiguous().view(-1, x.size(-1))
        y_ = y.contiguous().view(-1)
        loss = self.criterion(x_, y_)
        loss /= norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        #return loss.data[0] * norm  # TODO
        return loss.item() * norm

In [12]:
class Recognition_Dataset(object):

    def __init__(self, dataset_root_dir, lbl2id_map, sequence_len, max_ratio, phase='train', pad=0):

        if phase == 'train':
            self.img_dir = os.path.join(base_data_dir, 'train')
            self.lbl_path = os.path.join(base_data_dir, 'gt.txt')
        else:
            self.img_dir = os.path.join(base_data_dir, 'test')
            self.lbl_path = os.path.join(base_data_dir, 'test_gt.txt')
        self.lbl2id_map = lbl2id_map
        self.pad = pad   # padding标识符的id，默认0
        self.sequence_len = sequence_len    # 序列长度
        self.max_ratio = max_ratio * 3      # 将宽拉长3倍

        self.imgs_list = []
        self.lbls_list = []
        with open(self.lbl_path, 'r') as reader:
            for line in reader:
                items = line.rstrip().split(',')
                img_name = items[0]
                lbl_str = items[1].strip()[1:-1]

                self.imgs_list.append(img_name)
                self.lbls_list.append(lbl_str)

        # 定义随机颜色变换
        self.color_trans = transforms.ColorJitter(0.1, 0.1, 0.1)
        # 定义 Normalize
        self.trans_Normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
        ]) 

    def __getitem__(self, index):
        """ 
        获取对应index的图像和ground truth label，并视情况进行数据增强
        """
        img_name = self.imgs_list[index]
        img_path = os.path.join(self.img_dir, img_name)
        lbl_str = self.lbls_list[index]

        # ----------------
        # 图片预处理
        # ----------------
        # load image
        img = Image.open(img_path).convert('RGB')

        # 对图片进行大致等比例的缩放
        # 将高缩放到32，宽大致等比例缩放，但要被32整除
        w, h = img.size
        ratio = round((w / h) * 3)   # 将宽拉长3倍，然后四舍五入
        if ratio == 0:
            ratio = 1
        if ratio > self.max_ratio:
            ratio = self.max_ratio
        h_new = 32
        w_new = h_new * ratio
        img_resize = img.resize((w_new, h_new), Image.BILINEAR)

        # 对图片右半边进行padding，使得宽/高比例固定=self.max_ratio
        img_padd = Image.new('RGB', (32*self.max_ratio, 32), (0,0,0))
        img_padd.paste(img_resize, (0, 0))

        # 随机颜色变换
        img_input = self.color_trans(img_padd)
        # Normalize
        img_input = self.trans_Normalize(img_input)

        # ----------------
        # label处理
        # ----------------

        # 构造encoder的mask
        encode_mask = [1] * ratio + [0] * (self.max_ratio - ratio)
        encode_mask = torch.tensor(encode_mask)
        encode_mask = (encode_mask != 0).unsqueeze(0)

        # 构造ground truth label
        gt = []
        gt.append(1)    # 先添加句子起始符
        for lbl in lbl_str:
            gt.append(self.lbl2id_map[lbl])
        gt.append(2)
        for i in range(len(lbl_str), self.sequence_len):   # 除去起始符终止符，lbl长度为sequence_len，剩下的padding
            gt.append(0)
        # 截断为预设的最大序列长度
        gt = gt[:self.sequence_len+2]

        # decoder的输入
        decode_in = gt[:-1]
        decode_in = torch.tensor(decode_in)
        # decoder的输出
        decode_out = gt[1:]
        decode_out = torch.tensor(decode_out)
        # decoder的mask 
        decode_mask = self.make_std_mask(decode_in, self.pad)
        # 有效tokens数
        ntokens = (decode_out != self.pad).data.sum()

        return img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens 
    
    @staticmethod
    def make_std_mask(tgt, pad):
        """
        Create a mask to hide padding and future words.
        padd 和 future words 均在mask中用0表示
        """
        tgt_mask = (tgt != pad)
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        tgt_mask = tgt_mask.squeeze(0)   # subsequent返回值的shape是(1, N, N)
        return tgt_mask

    def __len__(self):
        return len(self.imgs_list)


# Model Architecture
class OCR_EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. 
    Base for this and many other models.
    """
    def __init__(self, encoder, decoder, src_embed, src_position, tgt_embed, generator):
        super(OCR_EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed    # input embedding module(input embedding + positional encode)
        self.src_position = src_position
        self.tgt_embed = tgt_embed    # ouput embedding module
        self.generator = generator    # output generation module
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        memory = self.encode(src, src_mask)
        res = self.decode(memory, src_mask, tgt, tgt_mask)
        return res
    
    def encode(self, src, src_mask):
        # feature extract
        src_embedds = self.src_embed(src)
        # 将src_embedds由shape(bs, model_dim, 1, max_ratio) 处理为transformer期望的输入shape(bs, 时间步, model_dim)
        src_embedds = src_embedds.squeeze(-2)
        src_embedds = src_embedds.permute(0, 2, 1)

        # position encode
        src_embedds = self.src_position(src_embedds)

        return self.encoder(src_embedds, src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        target_embedds = self.tgt_embed(tgt)
        return self.decoder(target_embedds, memory, src_mask, tgt_mask)


def make_ocr_model(tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    """
    构建模型
    params:
        tgt_vocab: 输出的词典大小(82)
        N: 编码器和解码器堆叠基础模块的个数
        d_model: 模型中embedding的size，默认512
        d_ff: FeedForward Layer层中embedding的size，默认2048
        h: MultiHeadAttention中多头的个数，必须被d_model整除
        dropout:
    """
    c = copy.deepcopy

    backbone = models.resnet18(pretrained=True)
    backbone = nn.Sequential(*list(backbone.children())[:-2])    # 去掉最后两个层 (global average pooling and fc layer)

    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)

    model = OCR_EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        backbone,
        c(position),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # Initialize parameters with Glorot / fan_avg.
    for child in model.children():
        if child is backbone:
            # 将backbone的权重设为不计算梯度
            for param in child.parameters():
                param.requires_grad = False
            # 预训练好的backbone不进行随机初始化，其余模块进行随机初始化
            continue
        for p in child.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    return model


def run_epoch(data_loader, model, loss_compute, device=None):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_loader):
        #if device == "cuda":
        #    batch.to_device(device) 
        img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens = batch
        img_input = img_input.to(device)                        
        encode_mask = encode_mask.to(device)                                
        decode_in = decode_in.to(device)                                
        decode_out = decode_out.to(device)                    
        decode_mask = decode_mask.to(device)
        ntokens = torch.sum(ntokens).to(device)

        out = model.forward(img_input, decode_in, encode_mask, decode_mask)

        loss = loss_compute(out, decode_out, ntokens)
        total_loss += loss
        total_tokens += ntokens
        tokens += ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens


# greedy decode
def greedy_decode(model, src, src_mask, max_len, start_symbol, end_symbol):
    memory = model.encode(src, src_mask)
    # ys代表目前已生成的序列，最初为仅包含一个起始符的序列，不断将预测结果追加到序列最后
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data).long()
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1)).type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        next_word = torch.ones(1, 1).type_as(src.data).fill_(next_word).long()
        ys = torch.cat([ys, next_word], dim=1)

        next_word = int(next_word)
        if next_word == end_symbol:
            break
        #ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    ys = ys[0, 1:]
    return ys


def judge_is_correct(pred, label):
    # 判断模型预测结果和label是否一致
    pred_len = pred.shape[0]
    label = label[:pred_len]
    is_correct = 1 if label.equal(pred) else 0
    return is_correct

In [13]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.model_size = model_size
        self._rate = 0
    
    def state_dict(self):
        """Returns the state of the warmup scheduler as a :class:`dict`.
        It contains an entry for every variable in self.__dict__ which
        is not the optimizer.
        """
        return {key: value for key, value in self.__dict__.items() if key != 'optimizer'}
    
    def load_state_dict(self, state_dict):
        """Loads the warmup scheduler's state.
        Arguments:
            state_dict (dict): warmup scheduler state. Should be an object returned
                from a call to :meth:`state_dict`.
        """
        self.__dict__.update(state_dict) 
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))

In [ ]:
# TODO set parameters
base_data_dir = '/content/drive/MyDrive/icdar15'    # 数据集根目录，请将数据下载到此位置
device = torch.device('cpu')
nrof_epochs = 300
batch_size = 64
model_save_path = './log/ex1_ocr_model.pth'

# 读取label-id映射关系记录文件
lbl2id_map_path = os.path.join(base_data_dir, 'lbl2id_map.txt')
lbl2id_map, id2lbl_map = load_lbl2id_map(lbl2id_map_path)

# 统计数据集中出现的所有的label中包含字符最多的有多少字符，数据集构造gt信息需要用到
train_lbl_path = os.path.join(base_data_dir, 'gt.txt')
valid_lbl_path = os.path.join(base_data_dir, 'test_gt.txt')
train_max_label_len = statistics_max_len_label(train_lbl_path)
valid_max_label_len = statistics_max_len_label(valid_lbl_path)
sequence_len = max(train_max_label_len, valid_max_label_len)   # 数据集中字符数最多的一个case作为制作的gt的sequence_len

# 构造 dataloader
max_ratio = 8    # 图片预处理时 宽/高的最大值，不超过就保比例resize，超过会强行压缩
train_dataset = Recognition_Dataset(base_data_dir, lbl2id_map, sequence_len, max_ratio, 'train', pad=0)
valid_dataset = Recognition_Dataset(base_data_dir, lbl2id_map, sequence_len, max_ratio, 'test', pad=0)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                    batch_size=batch_size,
                                    shuffle=False,
                                    num_workers=4)  

# build model
# use transformer as ocr recognize model
tgt_vocab = len(lbl2id_map.keys())
d_model = 512
ocr_model = make_ocr_model(tgt_vocab, N=5, d_model=d_model, d_ff=2048, h=8, dropout=0.1)
ocr_model.to(device)

# train prepare
criterion = LabelSmoothing(size=tgt_vocab, padding_idx=0, smoothing=0.0)

# choose a optimizer
optimizer = torch.optim.Adam(ocr_model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
#optimizer = torch.optim.SGD(ocr_model.parameters(), lr=0.001, momentum=0.9)
model_opt = NoamOpt(d_model,200,optimizer)


for epoch in range(nrof_epochs):
  print(f"\nepoch {epoch}")

  print("train...")
  ocr_model.train()
  loss_compute = SimpleLossCompute(ocr_model.generator, criterion, model_opt)
  #loss_compute = SimpleLossCompute(ocr_model.generator, criterion, optimizer)
  train_mean_loss = run_epoch(train_loader, ocr_model, loss_compute, device)

  if epoch % 20 == 0:
    print("valid...")
    ocr_model.eval()
    valid_loss_compute = SimpleLossCompute(ocr_model.generator, criterion, None)
    valid_mean_loss = run_epoch(valid_loader, ocr_model, valid_loss_compute, device)
    print(f"valid loss: {valid_mean_loss}")

# save model
torch.save(ocr_model.state_dict(), model_save_path)

# save model
torch.save(ocr_model.state_dict(), model_save_path)


epoch 0
train...
Epoch Step: 1 Loss: 4.850827 Tokens per Sec: 37.065262
Epoch Step: 51 Loss: 2.854465 Tokens per Sec: 42.415791
valid...
Epoch Step: 1 Loss: 2.871195 Tokens per Sec: 16.539686
valid loss: 2.7329490184783936

epoch 1
train...
Epoch Step: 1 Loss: 4.715374 Tokens per Sec: 36.040276
Epoch Step: 51 Loss: 2.817122 Tokens per Sec: 40.043793

epoch 2
train...
Epoch Step: 1 Loss: 2.563766 Tokens per Sec: 35.349197
Epoch Step: 51 Loss: 2.728527 Tokens per Sec: 39.267166

epoch 3
train...
Epoch Step: 1 Loss: 2.662421 Tokens per Sec: 31.443756
Epoch Step: 51 Loss: 2.544282 Tokens per Sec: 35.474258

epoch 4
train...
Epoch Step: 1 Loss: 2.590855 Tokens per Sec: 30.979532
Epoch Step: 51 Loss: 2.546010 Tokens per Sec: 35.175621

epoch 5
train...
Epoch Step: 1 Loss: 2.639744 Tokens per Sec: 32.460304
Epoch Step: 51 Loss: 2.644145 Tokens per Sec: 35.155739

epoch 6
train...
Epoch Step: 1 Loss: 2.601664 Tokens per Sec: 30.634779
Epoch Step: 51 Loss: 2.446588 Tokens per Sec: 36.092007

e

In [ ]:
# 训练结束，使用贪心的解码方式推理训练集和验证集，统计正确率
ocr_model.eval()
print("\n------------------------------------------------")
print("greedy decode trainset")
total_img_num = 0
total_correct_num = 0
for batch_idx, batch in enumerate(train_loader):
    img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens = batch
    img_input = img_input.to(device)                        
    encode_mask = encode_mask.to(device)                                

bs = img_input.shape[0]
for i in range(bs):
    cur_img_input = img_input[i].unsqueeze(0)
    cur_encode_mask = encode_mask[i].unsqueeze(0)
    cur_decode_out = decode_out[i]

    pred_result = greedy_decode(ocr_model, cur_img_input, cur_encode_mask, max_len=sequence_len, start_symbol=1, end_symbol=2)
    pred_result = pred_result.cpu()

    is_correct = judge_is_correct(pred_result, cur_decode_out)
    total_correct_num += is_correct
    total_img_num += 1
    if not is_correct:
        # 预测错误的case进行打印
        print("----")
        print(cur_decode_out)
        print(pred_result)
total_correct_rate = total_correct_num / total_img_num * 100
print(f"total correct rate of trainset: {total_correct_rate}%")

In [ ]:
print("\n------------------------------------------------")
print("greedy decode validset")
total_img_num = 0
total_correct_num = 0
for batch_idx, batch in enumerate(valid_loader):
    img_input, encode_mask, decode_in, decode_out, decode_mask, ntokens = batch
    img_input = img_input.to(device)                        
    encode_mask = encode_mask.to(device)                                

bs = img_input.shape[0]
for i in range(bs):
    cur_img_input = img_input[i].unsqueeze(0)
    cur_encode_mask = encode_mask[i].unsqueeze(0)
    cur_decode_out = decode_out[i]

    pred_result = greedy_decode(ocr_model, cur_img_input, cur_encode_mask, max_len=sequence_len, start_symbol=1, end_symbol=2)
    pred_result = pred_result.cpu()

    is_correct = judge_is_correct(pred_result, cur_decode_out)
    total_correct_num += is_correct
    total_img_num += 1
    if not is_correct:
        # 预测错误的case进行打印
        print("----")
        print(cur_decode_out)
        print(pred_result)
total_correct_rate = total_correct_num / total_img_num * 100
print(f"total correct rate of validset: {total_correct_rate}%")